In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
from scipy.optimize import fsolve
from numpy.linalg import inv

In [2]:
N=1000
alpha=0.05
z=norm.ppf(1-alpha/2)
pi=math.pi

In [3]:
n=50

X=[]
U=np.random.uniform(0,1,n)
for i in range(n):
    X.append((fsolve(lambda x: 0.5*norm.cdf(x)+0.5*norm.cdf(x-4)-U[i], 0)[0]))

In [4]:
def p1(X,mu1):
    return norm.pdf(X,mu1,1)

In [5]:
def p2(X,mu2):
    return norm.pdf(X,mu2,1)

In [6]:
def f(X,lamda,mu1,mu2):
    return (lamda*p1(X,mu1)+(1-lamda)*p2(X,mu2))

In [7]:
def ML(paramlist):
    lamda,mu1,mu2=paramlist[0],paramlist[1],paramlist[2]
    s1=0
    s2=0
    s3=0
    for i in range(n):
        s1=s1-(p1(X[i],mu1)-p2(X[i],mu2))/(f(X[i],lamda,mu1,mu2))
        s2=s2-lamda*p1(X[i],mu1)*(X[i]-mu1)/(f(X[i],lamda,mu1,mu2))
        s3=s3-(1-lamda)*p2(X[i],mu2)*(X[i]-mu2)/(f(X[i],lamda,mu1,mu2))
    return s1,s2,s3

In [8]:
lamda_hat,mu1_hat,mu2_hat=fsolve(ML,[0.5,0,4])

In [9]:
fsolve(ML,[0.5,0,4])

array([ 0.45089535, -0.19277317,  4.11826827])

In [10]:
h11=0
h12=0
h13=0
h22=0
h23=0
h33=0
for i in range(len(X)):
    h11=h11+(1/n)*(p1(X[i],mu1_hat)-p2(X[i],mu2_hat))**2/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
    h12=h12-(1/n)*(p1(X[i],mu1_hat)*(X[i]-mu1_hat)*f(X[i],lamda_hat,mu1_hat,mu2_hat)-p1(X[i],mu1_hat)*(X[i]-mu1_hat)*lamda_hat*(p1(X[i],mu1_hat)-p2(X[i],mu2_hat)))/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
    h13=h13-(1/n)*(-p2(X[i],mu2_hat)*(X[i]-mu2_hat)*f(X[i],lamda_hat,mu1_hat,mu2_hat)-p2(X[i],mu2_hat)*(X[i]-mu2_hat)*(1-lamda_hat)*(p1(X[i],mu1_hat)-p2(X[i],mu2_hat)))/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
    h22=h22-(1/n)*(lamda_hat*p1(X[i],mu1_hat)*(-1+(X[i]-mu1_hat)**2)*f(X[i],lamda_hat,mu1_hat,mu2_hat)-(p1(X[i],mu1_hat)*(X[i]-mu1_hat)*lamda_hat)**2)/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
    h33=h33-(1/n)*((1-lamda_hat)*p2(X[i],mu2_hat)*(-1+(X[i]-mu2_hat)**2)*f(X[i],lamda_hat,mu1_hat,mu2_hat)-(p2(X[i],mu2_hat)*(X[i]-mu2_hat)*(1-lamda_hat))**2)/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
    h23=h23+(1/n)*(p1(X[i],mu1_hat)*(X[i]-mu1_hat)*lamda_hat*p2(X[i],mu2_hat)*(X[i]-mu2_hat)*(1-lamda_hat))/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2

In [11]:
H=[[h11,h12,h13],[h12,h22,h23],[h13,h23,h33]]

In [12]:
H

[[3.7703860094021486, -0.14502157735705443, -0.14164070841456766],
 [-0.14502157735705443, 0.3699811516348409, -0.07387679976518237],
 [-0.14164070841456766, -0.07387679976518237, 0.4717990743449744]]

# Integration needs to be accurate

In [18]:
import scipy.integrate
a11= lambda x:(p1(x,mu1_hat)-p2(x,mu2_hat))**2/f(x,lamda_hat,mu1_hat,mu2_hat)
a22= lambda x:(lamda_hat*(x-mu1_hat)*p1(x,mu1_hat))**2/f(x,lamda_hat,mu1_hat,mu2_hat)
a33= lambda x:((1-lamda_hat)*(x-mu2_hat)*p2(x,mu2_hat))**2/f(x,lamda_hat,mu1_hat,mu2_hat)
a12= lambda x:(p1(x,mu1_hat)-p2(x,mu2_hat))*(lamda_hat*(x-mu1_hat)*p1(x,mu1_hat))/f(x,lamda_hat,mu1_hat,mu2_hat)
a13= lambda x:(p1(x,mu1_hat)-p2(x,mu2_hat))*((1-lamda_hat)*(x-mu2_hat)*p2(x,mu2_hat))/f(x,lamda_hat,mu1_hat,mu2_hat)
a23 = lambda x:(lamda_hat*(x-mu1_hat)*p1(x,mu1_hat))*((1-lamda_hat)*(x-mu2_hat)*p2(x,mu2_hat))/f(x,lamda_hat,mu1_hat,mu2_hat)
f11 = scipy.integrate.quad(a11, -35, 35)[0]
f22 = scipy.integrate.quad(a22, -35, 35)[0]
f33 = scipy.integrate.quad(a33, -35, 35)[0]
f12 = scipy.integrate.quad(a12, -35, 35)[0]
f13 = scipy.integrate.quad(a13, -35, 35)[0]
f23 = scipy.integrate.quad(a23, -35, 35)[0]

In [14]:
F=[[f11,f12,f13],[f12,f22,f23],[f13,f23,f33]]

In [15]:
F

[[3.847824678401031, -0.10046572943653717, -0.10354120900258094],
 [-0.10046572943653717, 0.3946992778776746, -0.05103756050901932],
 [-0.10354120900258094, -0.05103756050901932, 0.4896259182979119]]

In [16]:
inv(H)

array([[0.2739828 , 0.12781351, 0.1022672 ],
       [0.12781351, 2.84970161, 0.48459279],
       [0.1022672 , 0.48459279, 2.2261285 ]])

In [17]:
inv(F)

array([[0.26356444, 0.07530916, 0.06358605],
       [0.07530916, 2.58970873, 0.28587134],
       [0.06358605, 0.28587134, 2.0856207 ]])